In [1]:
## Import required Python modules
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy, scipy.stats
import io
import base64
#from IPython.core.display import display
from IPython.display import display, HTML, Image
from urllib.request import urlopen

try:
    import astropy as apy
    import astropy.table
    _apy = True
    #print('Loaded astropy')
except:
    _apy = False
    #print('Could not load astropy')

## Customising the font size of figures
plt.rcParams.update({'font.size': 14})

## Customising the look of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))
## This custom file is adapted from https://github.com/lmarti/jupyter_custom/blob/master/custom.include
HTML('custom.css')
#HTML(urlopen('https://raw.githubusercontent.com/bretonr/intro_data_science/master/custom.css').read().decode('utf-8'))

In [2]:
## Custom imports
from scipy.stats import binom, poisson, chi2, norm, uniform
from scipy.optimize import curve_fit
from math import ceil, pi
from numpy import exp
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle, Rectangle
from matplotlib.colors import makeMappingArray

ImportError: cannot import name 'makeMappingArray' from 'matplotlib.colors' (/Users/bretonr/.pyenv/versions/3.10.11/lib/python3.10/site-packages/matplotlib/colors.py)

In [ ]:
HTML('''
<script>
    function toggleCodeCells() {
      var codeCells = document.querySelectorAll('.jp-CodeCell');

      codeCells.forEach(function(cell) {
        var inputArea = cell.querySelector('.jp-InputArea');
        if (inputArea) {
          var currentDisplay = inputArea.style.display || getComputedStyle(inputArea).display;
          inputArea.style.display = currentDisplay === 'none' ? '' : 'none';
        }
      });
    }
</script>

<!-- Add a button to toggle visibility of input code cells -->
<button onclick="toggleCodeCells()">Toggle Code Cells</button>
''')

<div class="container-fluid">
    <div class="row">
        <div class="col-md-8" align="center">
            <h1>PHYS 10791: Introduction to Data Science</h1>
            <!--<h3>2019-2020 Academic Year</h3><br>-->
        </div>
        <div class="col-md-3">
            <img align='center' style="border-width:0" src="images/UoM_logo.png"/>
        </div>
    </div>
</div>

<div class="container-fluid">
    <div class="row">
        <div class="col-md-2" align="right">
            <b>Course instructors:&nbsp;&nbsp;</b>
        </div>
        <div class="col-md-9" align="left">
            <a href="http://www.renebreton.org">Prof. Rene Breton</a> - Twitter <a href="https://twitter.com/BretonRene">@BretonRene</a><br>
            <a href="http://www.hep.manchester.ac.uk/u/gersabec">Dr. Marco Gersabeck</a> - Twitter <a href="https://twitter.com/MarcoGersabeck">@MarcoGersabeck</a>
        </div>
    </div>
</div>

# Chapter 7 - Problem Sheet

### Problem 1: $\chi^2$ test

Tables to help with this problem are given below.

#### Problem 1.1:
You conduct a series $\chi^2$ tests resulting in $\chi^2$ values of 34, 37, and 40 for 25 degrees of freedom. Which of these results would you accept when checking against a probability of at least $5\%$ to exceed the $\chi^2$ value in question?

#### Problem 1.2:
You are conducting 50 experiments involving one $\chi^2$ test each. 
- If you conducted these tests with a $5\%$ probability as used in the previous problem, how many experiments would you expect to fail due to random fluctuations?
- What would a reasonable probability be for your $\chi^2$ tests to avoid random rejections of your experiments?
- If your experiments have 50 degrees of freedom, what would the largest acceptable $\chi^2$ value be?

In [ ]:
ndfs = np.array([1,2,3,4,5,6,7,8,9,10,15,20,25,30,40,50,75,100,150,200,300,500,750,1000])
chi2Ns = np.array([1.0,1.02,1.05,1.1,1.25,1.5,1.75,2.0,3.0,5.0,7.5,10.0])

np.set_printoptions(precision=2,suppress=True)
print('This table gives probabilities for exceeding the chi2/NDF value at the top of the column for the NDF at the beginning of the row.')
print('N \ chi2/N',chi2Ns)

np.set_printoptions(precision=3,suppress=True)
for n in ndfs:
    ps = []
    for c in chi2Ns:
        ps.append(1.-chi2.cdf(c*n, n))
    ps = np.array(ps)
    print('{0:10d}'.format(n),ps)
    
    
ps = np.array([0.2,0.1,0.05,0.01,0.001])
np.set_printoptions(precision=3,suppress=True)
print('\n\n\n')
print('This table gives chi2/NDF values the integral to which from 0 corresponds to the probability at the top of the column for the NDF at the beginning of the row.')
print('N \ prob',ps)

np.set_printoptions(precision=3,suppress=True)
for n in ndfs:
    cs = []
    for p in ps:
        cs.append(chi2.ppf(1.-p, n)/n)
    cs = np.array(cs)
    print('{0:8d}'.format(n),cs)

### Problem 2: Comparing fit models

Assess the following fit results according to both the Akaike and Bayesian Information Criteria. Determine the optimal model and judge which models would be acceptable at the $1\%$ and $10\%$ level.

#### Problem 2.1

You conduct a number of maximum likelihood fits to an unbinned dataset of 300 values, which have constant errors. The results are as follows:
- A fit with a parabola returns a maximum log likelihood value of $\ln{\mathcal L}=-145$.
- A fit with a constant returns a maximum log likelihood value of $\ln{\mathcal L}=-160$.
- A fit with the sum of a Gaussian distribution and a linear function returns a maximum log likelihood value of $\ln{\mathcal L}=-144$.

#### Problem 2.2

You conduct a number of $\chi^2$ fits to an unbinned dataset of 1000 values, which have constant errors. The results are as follows:
- A fit with an exponential function returns a minimum $\chi^2$ value of $\chi^2=1050$.
- A fit with the sum of an exponential function and a constant returns a minimum $\chi^2$ value of $\chi^2=980$.
- A fit with a fourth-order polynommial returns a minimum $\chi^2$ value of $\chi^2=970$.

### Problem 3: Two ensembles of measurements

Assume that we have two ensembles with a known spread and we want to test the compatibility of their means. An example of this scenario is your exams, which should neither be too easy nor too hard.

Let's assume one course has 310 students taking the exam and an average grade of $65.8\%$. Another course has 55 students who achieve on average $72.3\%$. Both distributions have a standard deviation of $8\%$.

Evaluate the statistical compatibility of the average grades. Discuss wheter this level of compatibility is acceptable. Are there any other aspects to be considered when making this comparison?

### Problem 4: Kolmogorov-Smirnov test

You are given the following test results for $D={\rm max}|{\rm cum}(x)-{\rm cum}(y)|$ alongside the sample sizes involved. Where only one sample size is given, the test reflects a comparison of a sample with a function, otherwise it refers to a two-sample comparison.

Evaluate whether you would accept or reject each sample at the $1\%$, $5\%$, and $10\%$ level according to the table given in the lecture.

| $D$ | $N_1$ | $N_2$ |
| :- | -: | :- |
|  0.256 | 30 | - |
|  0.058 | 450 | - |
|  0.249 | 30 | 300 |
|  0.233 | 100 | 100 |
|  0.066 | 1000 | 500 |

<div class="well" align="center">
    <div class="container-fluid">
        <div class="row">
            <div class="col-md-3" align="center">
                <img align="center" alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" width="60%">
            </div>
            <div class="col-md-8">
            This work is licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>).
            </div>
        </div>
    </div>
    <br>
    <br>
    <i>Note: The content of this Jupyter Notebook is provided for educational purposes only.</i>
</div>